# Extra sulfate and tris pH

In the open ocean, the total sulfate concentration has a close linear correlation with salinity. However, in riverine or estuarine environments, terrestrial sulfate sources may cause deviations from this trend. Here, we use Pytzer to calculate the effect of such deviations on the pH of tris in artificial "seawater" such as might be used to calibrate pH measurements in these environments.

## Select a Pitzer model

We begin by importing the Python modules that we will need, and defining the solution's composition and conditions:

In [1]:
from copy import deepcopy
import numpy as np
import pytzer as pz

# Select which Pitzer model parameter library to use
prmlib = deepcopy(pz.libraries.MarChemSpec)

## Solution composition and conditions

First, let's define which pH-conservative ions are present - that is, those not involved in the equilibria - and their concentrations. To start, we have the standard seawater composition of XYZ:

In [2]:
fixmols = np.array([0.44516, 0.05518, 0.01077, 0.01058, 0.56912]) # in mol/kg-H2O
fixions = np.array(['Na', 'Mg', 'Ca', 'K', 'Cl'])

Next, we define the equilibria that we want to consider, and the total concentrations of the relevant chemical species:

In [3]:
tots = np.array([0.08, 0.02926]) # in mol/kg-H2O
eles = np.array(['t_trisH', 't_HSO4'])

The final inputs define the solution conditions. We begin at 25 °C and 1 atmosphere:

In [4]:
tempK = np.array([298.15]) # temperature in K
pres = np.array([10.10325]) # pressure in dbar

## Preparations for Pytzer

We now need to do some pre-processing to prepare these and other inputs Pytzer's equilibrium solving functions. The first function, `getallions`, creates a list of all of the ions in the solution, including those with fixed concentrations and those involved in the equilibria. The second function, `assemble`, evaluates all the Pitzer model interaction coefficients at the input temperature and pressure that are needed to calculate activity coefficients for the solution.

In [5]:
allions = pz.properties.getallions(eles, fixions)
allmxs = pz.matrix.assemble(allions, tempK, pres, prmlib=prmlib)

In [6]:
eqstate_guess = [0.0, -16.5, 30.0]
lnks = np.array([
    pz.dissociation.trisH_BH64(tempK),
    pz.dissociation.HSO4_CRP94(tempK),
    pz.dissociation.H2O_MF(tempK),
])

eqstate = pz.equilibrate.solvequick(eqstate_guess, tots, fixmols, eles, allions, fixions, allmxs, lnks)
allmols, allions = pz.equilibrate.eqstate2mols(eqstate.x, tots, fixmols, eles, fixions)

mH, mHSO4 = [allmols[allions == ion] for ion in ['H', 'HSO4']]
pH_Free = -np.log10(mH)
pH_Total = -np.log10(mH + mHSO4)

print(pH_Free)
print(pH_Total)

[8.16930433]
[8.05759334]


Now add extra sulfate, charge-balanced by extra sodium cations:

In [7]:
fixmols_extra = deepcopy(fixmols)
tots_extra = deepcopy(tots)
extra_Na2SO4 = 0.02926
fixmols_extra[fixions == 'Na'] += 2*extra_Na2SO4
tots_extra[eles == 't_HSO4'] += extra_Na2SO4

eqstate_extra = pz.equilibrate.solvequick(eqstate.x, tots_extra, fixmols_extra, eles, allions, fixions, allmxs, lnks)
allmols_extra = pz.equilibrate.eqstate2mols(eqstate_extra.x, tots_extra, fixmols_extra, eles, fixions)[0]

mH, mHSO4 = [allmols_extra[allions == ion] for ion in ['H', 'HSO4']]
pH_Free = -np.log10(mH)
pH_Total = -np.log10(mH + mHSO4)

print(pH_Free)
print(pH_Total)

[8.16538829]
[7.97394375]


In [9]:
from scipy.misc import derivative

def getpH(eqstate_guess, tots, fixmols, eles, allions, fixions, allmxs, lnk_trisH, lnk_HSO4, lnk_H2O):
    lnks = np.array([lnk_trisH, lnk_HSO4, lnk_H2O])
    eqstate = pz.equilibrate.solvequick(eqstate_guess, tots, fixmols, eles, allions, fixions, allmxs, lnks)
    allmols = pz.equilibrate.eqstate2mols(eqstate.x, tots, fixmols, eles, fixions)[0]
    mH, mHSO4 = [allmols[allions == ion] for ion in ['H', 'HSO4']]
    pH_Free = -np.log10(mH)
    pH_Total = -np.log10(mH + mHSO4)
    return pH_Free, pH_Total

lnk_trisH = pz.dissociation.trisH_BH64(tempK)
lnk_HSO4 = pz.dissociation.HSO4_CRP94(tempK)
lnk_H2O = pz.dissociation.H2O_MF(tempK)
pH_Free, pH_Total = getpH(eqstate_guess, tots, fixmols, eles, allions, fixions, allmxs, lnk_trisH, lnk_HSO4, lnk_H2O)

d_pH_Total = derivative(lambda lnk_HSO4: getpH(eqstate_guess, tots, fixmols, eles, allions, fixions, allmxs,
    lnk_trisH, lnk_HSO4, lnk_H2O)[1], lnk_HSO4, dx=1e-5)

print(pH_Free)
print(pH_Total)

print(d_pH_Total)


[8.16930433]
[8.05759334]
[0.09850012]


In [ ]:
print(d_pH_Total)